# 4 sklearn监督学习
预加载实验所需的回归、分类数据及预定义功能函数：

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,mean_squared_error

# 自定义函数
## 评估回归
def EvaluateReg(x_train,x_test,y_train,y_test,model):
    pre_train = model.predict(x_train)
    pre_test = model.predict(x_test)
    mse_train = round(mean_squared_error(y_train,pre_train),2)
    mse_test = round(mean_squared_error(y_test,pre_test),2)
    r2_train = round(model.score(x_train,y_train),2)
    r2_test = round(model.score(x_test,y_test),2)
    print('train_r2:%s,train_mse:%s,\ntest_r2:%s,test_mse:%s.' % (r2_train,mse_train,r2_test,mse_test))

## 评估分类
def EvaluateClass(x_train,x_test,y_train,y_test,model):
    pre_train = model.predict(x_train)
    pre_test = model.predict(x_test)
    roc_train = round(roc_auc_score(y_train,pre_train),2)
    roc_test = round(roc_auc_score(y_test,pre_test),2)
    accraucy_train = round(model.score(x_train,y_train),2)
    accraucy_test = round(model.score(x_test,y_test),2)
    print('train_accraucy:%s,train_auc:%s,\ntest_accraucy:%s,test_auc:%s.' % (accraucy_train,roc_train,accraucy_test,roc_test))

# 实验数据
## 回归数据
dt1 = pd.read_csv('data/pp_gas_emission/gt_2011.csv')
dt2 = pd.read_csv('data/pp_gas_emission/gt_2012.csv')
dt3 = pd.read_csv('data/pp_gas_emission/gt_2013.csv')
dt4 = pd.read_csv('data/pp_gas_emission/gt_2014.csv')
train = pd.concat([dt1,dt2,dt3,dt4],axis=0).reset_index(drop=True)
test = pd.read_csv('data/pp_gas_emission/gt_2015.csv')
x_train1,y_train1,x_test1,y_test1 = train.iloc[:,:9],np.ravel(train.NOX),test.iloc[:,:9],np.ravel(test.NOX)
## 分类数据
data = pd.read_csv('data/default of credit card clients.csv')
x_train2,x_test2,y_train2,y_test2 = train_test_split(data.iloc[:,1:],data['default payment next month'],test_size=0.3,random_state=11)
y_train2,y_test2 = np.ravel(y_train2),np.ravel(y_test2)

# 数据预处理
## 回归数据
scaler = preprocessing.StandardScaler()
scaler.fit(x_train1)
x_train1 = scaler.transform(x_train1)
x_test1 = scaler.transform(x_test1)
## 分类数据
scaler.fit(x_train2)
x_train2 = scaler.transform(x_train2)
x_test2 = scaler.transform(x_test2)

## 4.1 广义线性模型
### 4.1.1 普通最小二乘法
处理[回归问题](https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD)，对特征有**非奇异性**（满秩、特征比数多）与**中心化/标准化**要求。

In [2]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,reg)

train_r2:0.51,train_mse:63.68,
test_r2:0.26,test_mse:91.84.


### 4.1.2 岭回归与分类
#### 4.1.2.1 岭回归
岭回归通过对系数的大小施加惩罚来解决普通最小二乘法对共线性敏感的问题，其最小化的是带L2正则化的残差平方和，L2正则系数 $\alpha$ 值越大，收缩量越大，对共线性的鲁棒性也越强。

In [3]:
from sklearn import linear_model
ridge = linear_model.Ridge(alpha=0.1, # 数值越大，惩罚越大
                           tol=0.001, # 预测精度
                           solver='auto', # auto,svd,cholesky,lsqr,sparse_cg,sag,saga
                           max_iter=None, # 共轭函数求解迭代次数,sparse_cg and lsqr
                           random_state=None) # 控制sag、saga
ridge.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,ridge)

train_r2:0.51,train_mse:63.68,
test_r2:0.26,test_mse:91.85.


由于岭回归对共线性的改善，所以可以使用岭迹图来判断是否剔除某参数以避免共线性。此外，ridge提供了便捷的交叉验证建模函数，进行快速定位：

In [11]:
ridgeCV = linear_model.RidgeCV(alphas=np.arange(0,10,0.5),
                               cv=5, # None：留一交叉验证，int：指定折叠数
                               scoring='r2') # 评估函数
ridgeCV.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,ridgeCV)
ridgeCV.alpha_ # ridgeCV.best_score_,ridgeCV.coef_,ridgeCV.intercept_

train_r2:0.51,train_mse:63.71,
test_r2:0.25,test_mse:92.7.


9.5

#### 4.1.2.2 岭分类
适用于[二分类问题](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)，模型将类别转换为 $-1,1$ 两种标签，然后使用回归的方式计算。                                              

In [9]:
ridgeClassiflier = linear_model.RidgeClassifier(alpha=0.1,
                                                tol=0.01,
                                                class_weight='balanced') # dict,balanced
ridgeClassiflier.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,ridgeClassiflier)

train_accraucy:0.69,train_auc:0.67,
test_accraucy:0.68,test_auc:0.67.


也有与之相匹配的快速交叉验证方法：

In [13]:
ridgeClassiflierCV = linear_model.RidgeClassifierCV(alphas=range(670,680,1),
                                                    cv=3,
                                                    scoring='roc_auc',
                                                    class_weight='balanced')
ridgeClassiflierCV.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,ridgeClassiflierCV)
ridgeClassiflierCV.alpha_

train_accraucy:0.69,train_auc:0.67,
test_accraucy:0.68,test_auc:0.67.


678

### 4.1.3 Lasso
Lasso通过最小化L1正则化的残差平方和，快速提取出重要变量，简化模型（使用LASSO回归系数轨迹）。

In [22]:
lasso = linear_model.Lasso(alpha=0.01, # 数值越大，惩罚越大
                           tol=0.01,
                           precompute=True) # 启用预定义的格拉姆矩阵加速计算
lasso.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,lasso)


train_r2:0.51,train_mse:63.76,
test_r2:0.25,test_mse:93.21.


In [23]:
lassoCV = linear_model.LassoCV(cv=3,
                               eps=0.01,
                               n_alphas=100, # alphas=[0.1,10]
                               tol=0.01)
lassoCV.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,lassoCV)

train_r2:0.5,train_mse:65.57,
test_r2:0.09,test_mse:113.02.


In [25]:
lassoLarsCV = linear_model.LassoLarsCV(cv=3,
                                       max_n_alphas=1000,
                                       eps=2.220446049250313e-16)
lassoLarsCV.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,lassoLarsCV)

train_r2:0.51,train_mse:63.8,
test_r2:0.24,test_mse:93.86.


In [26]:
lassoLarsIC = linear_model.LassoLarsIC(criterion='aic') # aic/bic
lassoLarsIC.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,lassoLarsIC)

train_r2:0.51,train_mse:63.68,
test_r2:0.26,test_mse:91.84.


与岭回归一样，Lasso也可以进行多任务分类：MultiTaskLasso。
### 4.1.4 Logistic回归
用于处理二分类问题，lbfgs求解器鲁棒性占优；对于大型数据集，saga求解器通常更快。

In [27]:
lr = linear_model.LogisticRegression(C=10000, # 正则化系数的倒数
                                     #class_weight='balanced',
                                     penalty='elasticnet', # l1,l2,elasticnet;l1-liblinear
                                     solver='saga', # liblinear(坐标轴下降法),lbfgs(loss二阶导),newton-cg(loss二阶导),sag(随机平均梯度下降),saga
                                     tol=0.01, # 迭代终止判据的误差范围
                                     max_iter=500,
                                     random_state=11, # sag,saga,liblinear时
                                     l1_ratio=0.9) # 仅在惩罚='elasticnet'时使用；值为0等同于使用惩罚='l2'，值为1等同于使用惩罚='l1'。当0 < l1_ratio <1时，罚点球是L1和L2的组合。

lr.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,lr)

train_accraucy:0.81,train_auc:0.61,
test_accraucy:0.81,test_auc:0.61.


In [29]:
lrCV = linear_model.LogisticRegressionCV(Cs=[0.1,1,10,100,1000,10000],
                                         cv=3,
                                         #class_weight='balanced',
                                         penalty='elasticnet', # l1,l2,elasticnet
                                         scoring='roc_auc',
                                         solver='saga', # newton-cg,lbfgs,liblinear,sag,saga
                                         tol=0.01,
                                         max_iter=500,
                                         random_state=11,
                                         l1_ratios=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
lrCV.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,lrCV)

train_accraucy:0.72,train_auc:0.61,
test_accraucy:0.73,test_auc:0.61.


一点感悟：balanced调参，非balanced训练最终模型，效果比较好。
### 4.1.6 随机梯度下降
适合大数据集，设定 loss="log" ，则 SGDClassifier 拟合一个逻辑回归模型，而 loss="hinge" 拟合线性支持向量机（SVM）。

In [34]:
sgd = linear_model.SGDRegressor(loss='epsilon_insensitive', # squared_loss,huber(对异常不敏感),epsilon_insensitive(超限后线性),squared_epsilon_insensitive
                                penalty='elasticnet', # l2,l1,elasticnet
                                alpha=0.0001, # 值越大，正则化越强;optimal
                                l1_ratio=0.5,
                                max_iter=100,
                                tol=0.01,
                                epsilon=0.1, # huber,epsilon_insensitive,squared_epsilon_insensitive 敏感阈值界限
                                random_state=11,
                                learning_rate='optimal', # constant,optimal,invscaling,adaptive
                                eta0=0.1, # 'constant','invscaling','adaptive'的初始化学习率
                                # power_t=0.25, # invscaling 所需
                                early_stopping=True,
                                validation_fraction=0.1, # early_stopping预留验证集比例
                                n_iter_no_change=50)
sgd.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,sgd)

train_r2:0.43,train_mse:74.59,
test_r2:0.19,test_mse:99.83.


In [38]:
sgd = linear_model.SGDClassifier(loss='hinge', # hinge,log,modified_huber,squared_hinge,perceptron;squared_loss,huber,epsilon_insensitive,squared_epsilon_insensitive.
                                 penalty='elasticnet',
                                 alpha=0.2,
                                 l1_ratio=0.2,
                                 random_state=11,
                                 learning_rate='invscaling',
                                 eta0=0.1,
                                 power_t=0.4,
                                 early_stopping=True,
                                 validation_fraction=0.1,
                                 n_iter_no_change=100,
                                 class_weight='balanced')
sgd.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,sgd)

train_accraucy:0.81,train_auc:0.68,
test_accraucy:0.8,test_auc:0.69.


### 4.1.7 多项式回归
构建多项式、交互特征，拓展特征矩阵后建模

In [47]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
model = Pipeline([('poly',PolynomialFeatures(degree=3,interaction_only=False)),('sgd',linear_model.SGDClassifier(loss='hinge',penalty='elasticnet',alpha=0.2,l1_ratio=0.2))])
model.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,model)


train_accraucy:0.81,train_auc:0.59,
test_accraucy:0.8,test_auc:0.58.


### 4.1.8 稳健回归
稳健回归（robust regression）特别适用于回归模型包含损坏数据（corrupt data）的情况，如离群点或模型中的错误；但在高维数据条件下（ n_features大），一般而言很难完成稳健拟合，很可能完全不起作用。
## 4.2 线性和二次判别分析
线性判别分析（LDA）和二次判别分析（QDA）是有监督的数据降维思想，不同于后面要提及的主成分分析方法（PCA），属于无监督方法；所以LDA与QDA即可降维亦可分类。
### 4.2.1. 线性判别分析
LDA的思想是将数据投影到低维空间后使得同一类数据尽可能的紧凑，不同类的数据尽可能分散，适用于不同类别间数据均值差异很大，方差差异不大的情景。实际中，即使不满足上述要求，LDA一般也可取得很好的效果。

In [52]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(solver='eigen', # svd:适合特征多,lsqr,eigen
          shrinkage=0.5, # [0,1];lsqr/eigen & covariance_estimator=None
          n_components=1) # 降维的维数；用于分类则置None。
lda.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,lda)

train_accraucy:0.81,train_auc:0.64,
test_accraucy:0.81,test_auc:0.65.


### 4.2.2 二次判别分析
不同于LDA，QDA利用贝叶斯规则拟合数据的条件密度，生成二次决策边界。

In [53]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
qda = QDA().fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,qda)

train_accraucy:0.53,train_auc:0.63,
test_accraucy:0.52,test_auc:0.63.


## 4.3 内核岭回归

In [56]:
from sklearn import kernel_ridge
clf = kernel_ridge.KernelRidge(alpha=0.1, # 正则系数
                               kernel='linear') # linear,rbf,sigmoid,poly/polynomial,laplacian,cosine,chi2,additive_chi2
                               # gamma,仅rbf,laplacian,poly,chi2,sigmoid
                               # degree,仅poly
                               # coef0,仅poly、sigmoid核
clf.fit(x_train1,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,clf)


train_accraucy:-0.16,train_auc:0.72,
test_accraucy:-0.15,test_auc:0.73.


## 4.4 支持向量机
- 高维空间中高效
- features大于samples下仍有效
- 注意正则和核函数避免过拟合

In [2]:
from sklearn import svm
clf = svm.SVR(C=1.0,# 与惩罚呈反比
              kernel='poly', # linear,poly,rbf,sigmoid,precomputed
              degree=3) # poly
              # gamma='scale',rbf/poly/sigmoid
              # coef0=0.0,poly/sigmoid
clf.fit(x_train1,y_train1)
EvaluateReg(x_train1,x_test1,y_train1,y_test1,clf)


train_r2:0.62,train_mse:50.07,
test_r2:-0.33,test_mse:164.6.


In [65]:
clf = svm.SVC(C=1.0,# 与惩罚呈反比
              kernel='poly', # linear,poly,rbf,sigmoid,precomputed
              degree=3, # poly
              # gamma='scale',rbf/poly/sigmoid
              # coef0=0.0,poly/sigmoid
              decision_function_shape='ovr', # ovo / ovr
              random_state=None)
clf.fit(x_train2,y_train2)
EvaluateClass(x_train2,x_test2,y_train2,y_test2,clf)

train_accraucy:0.82,train_auc:0.61,
test_accraucy:0.8,test_auc:0.6.


## 4.5 最近邻
### 4.5.1 无监督最近邻
用于探测数据点的距离关系。

In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2,
                        radius=0.2, # 距离半径
                        algorithm='auto', # ball_tree,kd_tree,brute
                        leaf_size=30, # BallTree(克服KD树高维失效) / KDTree(维数小于20时效率最高)
                        metric='minkowski', # 树距离的度量方法
                        p=2, # p=1：曼哈顿距离，p=2，欧氏距离
                        n_jobs=-1).fit(X)
distances,indices = nbrs.kneighbors(X)
nbrs.kneighbors_graph(X).toarray() # 距离相同会按数据顺序选取前k个

array([[1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1.]])

### 4.5.2 有监督最近邻
- KNN：较常用，通常较大的 $k$ 会抑制噪声的影响，但使得分类界限不明显。
- RadiusNeighborsClassifier：当数据不均匀，基于半径的近邻分类可能是更好的选择；对于高维空间，由于“维度灾难”而不那么有效。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:


## 4.7 高斯过程
## 4.8 横向分解
## 4.9 朴素贝叶斯
## 4.10 决策树
## 4.11 整体方法
## 4.12 多类和多输出算法
## 4.13 特征选择
## 4.14 Semi-supervised学习
## 4.15 等张回归
## 4.16 概率校准
## 4.17 神经网络模型(监督)

## 过拟合


不同于PCA方差最大化理论，
PCA(principal Component Analysis)，即(无监督))，是一种使用最广泛的数据压缩算法。在PCA中，数据从原来的坐标系转换到新的坐标系，由数据本身决定。转换坐标系时，以方差最大的方向作为坐标轴方向，因为数据的最大方差给出了数据的最重要的信息。第一个新坐标轴选择的是原始数据中方差最大的方法，第二个新坐标轴选择的是与第一个新坐标轴正交且方差次大的方向。重复该过程，重复次数为原始数据的特征维数。

通过这种方式获得的新的坐标系，我们发现，大部分方差都包含在前面几个坐标轴中，后面的坐标轴所含的方差几乎为0,。于是，我们可以忽略余下的坐标轴，只保留前面的几个含有绝不部分方差的坐标轴。事实上，这样也就相当于只保留包含绝大部分方差的维度特征，而忽略包含方差几乎为0的特征维度，也就实现了对数据特征的降维处理。